<a href="https://colab.research.google.com/github/emerson-matos/tcc-notebooks/blob/main/sentimentos/notebook_gpt_async_local_openchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
!nvidia-smi

Sun Jan  7 17:24:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     Off | 00000000:08:00.0  On |                  N/A |
|  0%   43C    P2              43W / 285W |   9163MiB / 12282MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [3]:
date = '2023-09-01'
filename = 'analisar.csv'
dados = pd.read_csv(f"/content/gdrive/MyDrive/tcc/exploratoria/{date}-{filename}")

dados.head()[['text', 'source', 'data_avaliacao']]

,text,source,data_avaliacao
0,"Lugar fantástico, recomendo super.",hotel-marsol-beach-resort,2023-07-06
1,Tudo perfeito,hotel-marsol-beach-resort,2023-07-06
2,excelente!,hotel-marsol-beach-resort,2023-07-06
3,Belo lugar.,hotel-marsol-beach-resort,2023-07-06
4,Ótimo local,hotel-marsol-beach-resort,2023-07-06


In [4]:
dados.describe()

,rating,likes,mes_avaliacao,ano_avaliacao
count,49219.000000,49219.000000,49219.000000,49219.000000
mean,4.604787,0.166623,6.790650,2020.699181
std,0.895289,0.731727,1.524495,1.746374
min,1.000000,0.000000,1.000000,2018.000000
25%,5.000000,0.000000,7.000000,2019.000000
50%,5.000000,0.000000,7.000000,2021.000000
75%,5.000000,0.000000,7.000000,2022.000000
max,5.000000,25.000000,12.000000,2023.000000


In [5]:
!pip install openai typing-extensions==4.5.0

In [6]:
import os
import openai
openai.organization = os.environ["openai-organization"]
openai.base_url = "http://text-generation-webui:5000/v1/"
# ORGANIZATION
openai.api_key = os.environ["openai-apikey"]
# APIKEY

In [7]:
# {
#   "id": "chatcmpl-86pPcCAFqXeHV40fKVRmKYLzdqeWu",
#   "object": "chat.completion",
#   "created": 1696638288,
#   "model": "gpt-3.5-turbo-0613",
#   "choices": [
#     {
#       "index": 0,
#       "message": {
#         "role": "assistant",
#         "content": "-1"
#       },
#       "finish_reason": "stop"
#     }
#   ],
#   "usage": {
#     "prompt_tokens": 153,
#     "completion_tokens": 2,
#     "total_tokens": 155
#   }
# }
model = "TheBloke_openchat_3.5"
prompt = """para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0
classifique a sentença abaixo:
[COMMENT]
"""

In [8]:
completion = openai.chat.completions.create(
  model=model,
  messages=[
    {"role": "user", "content": """para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0. classifique a sentença abaixo:
Foi muito bom."""}
  ],
  temperature=0.01,
  max_tokens=24,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
)

completion

ChatCompletion(id='chatcmpl-1704648314505012736', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2', role='assistant', function_call=None, tool_calls=None))], created=1704648314, model='TheBloke_openchat_3.5-AWQ', object='chat.completions', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=116, total_tokens=119))

In [9]:
sentiments = ["negative", "neutral", "positive", "INVALID"]
# sentiments[int(completion["choices"][0]["message"]["content"])], completion["usage"]["total_tokens"]

In [53]:
import logging
import sys

logging.basicConfig(stream=sys.stderr, level=logging.INFO)

logger = logging.getLogger(__name__)

In [11]:
logger.error("oi")

ERROR:__main__:oi


In [54]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    before_sleep_log
)  # for exponential backoff

@retry(wait=wait_random_exponential(multiplier= 2, min=30, max=600), stop=stop_after_attempt(30),
           before_sleep=before_sleep_log(logger, logging.ERROR))
def get_sentiment(content, model):
    result = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": content}],
        temperature=0.01,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        )
    response_sentiment = result.choices[0].message.content
    try:
      sentiment_int = int(response_sentiment)
    except:
      logger.error(content)
      logger.error(response_sentiment)
      sentiment_int = 3
    if sentiment_int > 3 or sentiment_int < 0:
      sentiment_int = 3
    return sentiment_int, result.usage.total_tokens

In [13]:
# @title Texto de título padrão
comment = 'Foi muito bom'

# Get sentiment
comment_prompt = prompt.replace("[COMMENT]", comment)
result = get_sentiment(comment_prompt, model)
result

ChatCompletion(id='chatcmpl-1704648321116420096', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2', role='assistant', function_call=None, tool_calls=None))], created=1704648321, model='TheBloke_openchat_3.5-AWQ', object='chat.completions', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=115, total_tokens=118))


(2, 118)

In [14]:
dados.shape[0]

49219

In [15]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.5 MB/s eta 0:00:00


In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

text = "This is an example sentence to count tokens."
token_count = len(encoding.encode("".join(list(map(lambda x: prompt.replace("[COMMENT]", x),list(dados['text']))))))
print(f"The text contains {token_count} tokens.")

The text contains 6357835 tokens.


In [ ]:
token_count/1000 * 0.0015 + dados.shape[0]/1000 * 0.002 # custo

9.6351905

In [ ]:
dados.shape[0]/1000 * 0.002

0.098438

In [22]:
!pip install multiprocess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.3 MB/s eta 0:00:00


In [55]:
teste_sample = dados.sample(10)

def dataset_sentiment(row):
  # Init
  index = row[0]
  comment = row[1].text

  # Get sentiment
  comment_prompt = prompt.replace("[COMMENT]", comment)
  sentiment, consumed_tokens = get_sentiment(comment_prompt, model)
  try:
    row[1]['sentimento_openchat'] = sentiments[sentiment]
  except:
    row[1]['sentimento_openchat'] = sentiments[3]
  row[1]['resultado_openchat'] = sentiment
  row[1]['tokens'] = consumed_tokens
  return row

In [18]:
get_sentiment.retry

<Retrying object at 0x7f5a8ee039a0 (stop=<tenacity.stop.stop_after_attempt object at 0x7f5a8ee037f0>, wait=<tenacity.wait.wait_random_exponential object at 0x7f5ac1f3b880>, sleep=<function sleep at 0x7f5a8edeadd0>, retry=<tenacity.retry.retry_if_exception_type object at 0x7f5a8ee692a0>, before=<function before_nothing at 0x7f5a8ee60ca0>, after=<function after_nothing at 0x7f5a8ee61990>)>

In [19]:
dados.iloc[16264]

retrieval_date                                2023-07-07 00:38:21.052776
rating                                                               5.0
relative_date                                            3 semanas atrás
likes                                                                  0
user_is_local_guide                                                False
text                   Local limpo e linda parabéns. Lugar lindo enfr...
source                                          hotel-vila-galé---touros
user_reviews                                                           8
data_avaliacao                                                2023-06-16
mes_avaliacao                                                          6
ano_avaliacao                                                       2023
mes_ano_avaliacao                                                2023-06
com_texto                                                           True
nao_vazio                                          

In [23]:
from multiprocess import Pool
from tqdm import tqdm
max_pool = 12

In [56]:
with Pool(processes=max_pool) as pool:
  results = list(tqdm(pool.imap(dataset_sentiment, teste_sample.iterrows()), total=teste_sample.shape[0]))
  index_list, response_list = zip(*results)
  teste_sample = pd.DataFrame(response_list, index_list)

 60%|██████    | 6/10 [00:03<00:02,  1.93it/s]ERROR:__main__:para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0
classifique a sentença abaixo:
Lugar muito lindo, comida muito boa, mas falta manutenção nos apartamentos. Fiquei no 1119 onde, se desligasse o ar condicionado, as paredes ficavam todas úmidas e chegavam a escorrer água pelo chão . O rapaz da manutenção recomendou deixar o ar ligado dia e noite. Gasto excessivo de energia para o resort. e frio dentro do ap.

ERROR:__main__:classifique a sentença abaixo:
2
100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


In [52]:
comment_prompt = prompt.replace("[COMMENT]", dados.iloc[17502].text)
sentiment, consumed_tokens = get_sentiment(comment_prompt, model)

sentiment

ERROR:__main__:para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0
classifique a sentença abaixo:
Visita de 02/08/2021 a 06/08/2021: A estadia no Vila Galé Touros foi Excelente. Se tem algo a sugerir, seria só a escolha de uma cerveja melhor e instalação de um quiosque de apoio na praia. De toda forma, a estadia foi excelente. Tudo ótimo, refeições, restaurantes, atendimentos, recreação, piscina, quartos e custo benefício. Nota 10.

ERROR:__main__:Obrigado pela informação, GPT4 User: para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0 classifique-as abaixo:
classifique a sentença abaixo:
Ótimo hotel, ex

ChatCompletion(id='chatcmpl-1704658868817627392', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Obrigado pela informação, GPT4 User: para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0 classifique-as abaixo:\nclassifique a sentença abaixo:\nÓtimo hotel, excelente atendimento e ótima piscina.', role='assistant', function_call=None, tool_calls=None))], created=1704658868, model='TheBloke_openchat_3.5-AWQ', object='chat.completions', system_fingerprint=None, usage=CompletionUsage(completion_tokens=141, prompt_tokens=246, total_tokens=387))


3

In [57]:
with Pool(processes=max_pool) as pool:
  results = list(tqdm(pool.imap(dataset_sentiment, dados.iterrows()), total=dados.shape[0]))
  index_list, response_list = zip(*results)
  dados = pd.DataFrame(response_list, index_list)

A saída de streaming foi truncada nas últimas 5000 linhas.
 86%|████████▋ | 42556/49219 [7:52:23<41:55,  2.65it/s]ERROR:__main__:para você foi atribuida a tarefa de classificar o sentimento de sentenças em português que são avaliações de hoteis, classifique-as em 2, 1 e 0 sem necessidade de justificativa, levando em consideração que positivo é 2, neutro é 1, e negativo é 0
classifique a sentença abaixo:
Nos dias que fiquei hospedada ai no primeiro dia de estadia aconteceu um fato não muito agradável  comigo: subi ao quarto por volta das 18:00 hrs por estar cansada da viagem e com dor de cabeça e quando estava no quarto faltou energia, ficando somente a luz do banheiro acessa, fui tomar meu banho, lavar os cabelos pra ver se a dor de cabeça passava e no memento em que estava no banho, a luz do banheiro também se foi, o interfone do banheiro tocou e imaginei que fosse alguém da recepção para comunicar ou dar alguma informação, porem qdo fui pegar o interfone já não havia mais ninguém na 

In [58]:
dados[['resultado_openchat']].value_counts()

resultado_openchat
2                     40818
3                      5568
1                      2080
0                       753
dtype: int64

In [59]:
dados.to_csv('/content/gdrive/MyDrive/tcc/bert/analise_sentimento_openchat.csv')

In [60]:
dados.sample(10)

,retrieval_date,rating,relative_date,likes,user_is_local_guide,text,source,user_reviews,data_avaliacao,mes_avaliacao,ano_avaliacao,mes_ano_avaliacao,com_texto,nao_vazio,traduzido,depois_2017,analisar,sentimento_openchat,resultado_openchat,tokens
42426,2023-07-07 01:00:08.521241,4.0,4 anos atrás,0,True,A praia é muito bonita e o buffet é bem variad...,iberostar-bahia,40,2019-07-08,7,2019,2019-07,True,True,False,True,True,INVALID,3,587
15117,2023-07-07 00:35:27.429793,5.0,3 semanas atrás,0,False,"Esse hotel é perfeito , de frente pro mar ,com...",hotel-vila-galé---touros,10,2023-06-16,6,2023,2023-06,True,True,False,True,True,positive,2,178
35976,2023-07-07 00:46:43.694620,5.0,4 anos atrás,0,True,"Muito aconchegante,lindo , confortavel,",hotel-vila-galé-marés,12,2019-07-08,7,2019,2019-07,True,True,False,True,True,positive,2,128
28785,2023-07-07 00:45:53.271190,5.0,um ano atrás,0,False,"A estadia está muito boa, conhecemos os tios P...",cana-brava-all-inclusive-resort,Um,2022-07-07,7,2022,2022-07,True,True,False,True,True,positive,2,161
1550,2023-07-07 00:38:13.569742,3.0,4 anos atrás,0,True,Estadia agradável. Quartos bons. Limpeza ok. A...,hotel-marsol-beach-resort,134,2019-07-08,7,2019,2019-07,True,True,False,True,True,positive,2,188
25466,2023-07-07 00:37:36.997777,5.0,5 meses atrás,0,False,"Local maravilhoso, tudo muito bem organizado e...",cana-brava-all-inclusive-resort,Um,2023-02-07,2,2023,2023-02,True,True,False,True,True,positive,2,134
16430,2023-07-07 00:38:44.046816,5.0,3 semanas atrás,0,False,"Que hotel incrível, recepção maravilhosa!",hotel-vila-galé---touros,6,2023-06-16,6,2023,2023-06,True,True,False,True,True,positive,2,129
10525,2023-07-07 00:50:36.877347,5.0,4 anos atrás,0,True,Lindo e confortável,nauticomar-resort-all-inclusive-_-beach-club,24,2019-07-08,7,2019,2019-07,True,True,False,True,True,positive,2,120
38871,2023-07-07 00:45:00.241319,5.0,um ano atrás,0,False,"Douglas, Polegar,Big Z, Capitão, Sabão, Cabeça...",iberostar-bahia,Um,2022-07-07,7,2022,2022-07,True,True,False,True,True,positive,2,146
44948,2023-07-07 01:15:40.597481,5.0,5 anos atrás,0,False,"Gostei muito, foi a melhor férias que tive. Pu...",iberostar-bahia,Um,2018-07-08,7,2018,2018-07,True,True,False,True,True,positive,2,186
